# ライブラリのimport
Pytorch，Transformers，Juman++．

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from pyknp import Juman

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# モデルとTokenizerの読み込み
[京都大学，黒橋・褚・村脇研究室の日本語で学習したBERTの事前学習モデル](https://huggingface.co/nlp-waseda/roberta-base-japanese)と，TransformersライブラリのTokenizerを読み込む．

In [2]:
tokenizer = AutoTokenizer.from_pretrained("nlp-waseda/roberta-base-japanese")
model = AutoModelForMaskedLM.from_pretrained("nlp-waseda/roberta-base-japanese")
# model.cuda() # GPU対応

Downloading: 100%|██████████| 431/431 [00:00<00:00, 505kB/s]
Downloading: 100%|██████████| 810k/810k [00:00<00:00, 2.47MB/s]
Downloading: 100%|██████████| 244/244 [00:00<00:00, 268kB/s]
/usr/local/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Downloading: 100%|██████████| 637/637 [00:00<00:00, 635kB/s]
Downloading: 100%|██████████| 443M/443M [01:50<00:00, 4.02MB/s] 


# 形態素解析
Juman++を使用．
「りんごが宙を舞う。」

In [11]:
# Juman++
jumanpp = Juman()

sentence = "りんごが宙を舞う。"
result = jumanpp.analysis(sentence)
tokens = [mrph.midasi for mrph in result.mrph_list()]
print(tokens)

['りんご', 'が', '宙', 'を', '舞う', '。']


# マスキング
文章の一部をマスク．

In [13]:
# マスキング
masked_index = 2
tokens[masked_index] = '[MASK]'
print(tokens)

# マスク後の文章
masked_sentence = ' '.join(tokens)
print(masked_sentence)

['りんご', 'が', '[MASK]', 'を', '舞う', '。']
りんご が [MASK] を 舞う 。


# Tokenize
各形態素のindexを取得．

In [37]:
# tokenize
tokenized = tokenizer(masked_sentence, return_tensors='pt')
x = tokenized['input_ids']
print(x)

# masked_tokens = tokenizer.convert_ids_to_tokens(x[0].tolist())
# print(masked_tokens)

tensor([[    2, 27643,   268,     4,   266,  5251,   906,   264,     3]])
['[CLS]', '▁りんご', '▁が', '[MASK]', '▁を', '▁舞', 'う', '▁。', '[SEP]']


# 推論

In [39]:
# モデル
y = model(x)

# 出力結果
predictions = y[0]
masked_index = 3
_, predicted_indexes = torch.topk(predictions[0, masked_index], k=10)
print(predicted_indexes)

tensor([ 5251, 16105,   538,  1937,   340,  3351,  6874,   389,   431, 20254])


# 推測された単語
sentencepieceライブラリの仕様上，先頭に_が付く．

In [40]:
# マスクされた単語の推測
predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_indexes.tolist())
print(predicted_tokens)

['▁舞', '▁宙', '▁中心', '▁空', '▁大', '▁一番', '▁空中', '▁世界', '▁お', '▁まわり']
